In [ ]:
# Mounting google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Importing libraries
import numpy as np
import os
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Sequential, layers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.layers import Rescaling
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Lambda

In [ ]:
# This is the function to plot the accuracy and loss
# Plot the training history
def summarize_diagnostics(history):
    plt.figure(figsize=(10, 4))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='val')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Set the path to your dataset folder
data_dir = '/content/drive/MyDrive/Data_challenge/train/dataset_1'

# Define the image size and batch size
img_height, img_width = 512, 640
batch_size = 16

In [ ]:

# Load the pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in resnet.layers:
    layer.trainable = False

# Convert grayscale to RGB
def grayscale_to_rgb(image):
    return tf.image.grayscale_to_rgb(image)

# Add new layers for your own dataset
inputs = tf.keras.Input(shape=(img_height, img_width, 1))
rgb_images = Lambda(grayscale_to_rgb)(inputs)
x = resnet(rgb_images)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Create a new model
model = Model(inputs=inputs, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Loading the data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  label_mode='categorical',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  label_mode='categorical',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale'
)

# Train the model
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=25,
  batch_size=batch_size,
  callbacks=[earlystop_callback]
)


94765736/94765736 [==============================] - 5s 0us/step
Found 200 files belonging to 2 classes.
Using 160 files for training.
Found 200 files belonging to 2 classes.
Using 40 files for validation.
Epoch 1/25
10/10 [==============================] - ETA: 0s - loss: 72.2835 - accuracy: 0.4875

In [ ]:
summarize_diagnostics(history)

In [ ]:
model.save('/content/drive/MyDrive/Img_for_segmentation/resnet50_v2.h5')